# Mock Data
- Take the Theta file
- predict spec and phot
- distort spec and phot with a gaussian sigma=phot/snr
- snr has to be smaller than 40

In [1]:
import os
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from astropy.io import fits

from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.io import write_results as writer
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis
from prospect.utils.obsutils import fix_obs

import prospect.io.read_results as reader
import toolbox_prospector as toolbox

from sedpy.observate import load_filters

import Build


In [2]:
iot_number = '8'

## Read in all the data

In [3]:
path_wdir   =   "/Users/amanda/Desktop/Paper/technical/"
path_data   =   os.path.join(path_wdir, "data/")
path_plots  =   os.path.join(path_wdir, 'plots/iot_'+iot_number+'/')
path_output =   os.path.join(path_wdir, 'prospector/input_output_test'+iot_number+'/')
path_flury  =   os.path.join(path_data, 'flury/')
path_mock   =   os.path.join(path_data, 'mock/')


#------------------------------------------------------Emission Lines-----------------------------------------------------------------------

el_table  =  "lzlcs_optlines_obs.csv"
el        =   Table.read(os.path.join(path_flury, el_table), format="ascii")

el.add_column([i for i in range(1,67)], name='id', index=0) #uniform reference for name now possible


#-------------------------------------------------------Photometry--------------------------------------------------------------------------

phot_table  =   'GP_Aperture_Matched_Photometry_v0.fits'
phot_cat    =   fits.open(os.path.join(path_flury, phot_table))
phot        =   phot_cat[1].data


#------------------------------------------------------Signal to Noise----------------------------------------------------------------------


snr      = pd.read_csv(path_data + 'snr.csv')
snr_phot = snr['median'][:7] 
snr_el   = snr['median'][7:]


In [4]:
str(0.9)

'0.9'

In [5]:
thetas = {}

thetas['parnames']        =  ['logzsol', 'dust2', 'logmass', 'logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios', 'dust_index', 'dust1_fraction', 'igm_factor', 'gas_logz', 'gas_logu', 'frac_obrun']
thetas['thetas_original'] =  [-0.5,  0.6, 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. ,  0. , -2. ,  0. ]
thetas['thetas_0.0']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0. ]
thetas['thetas_0.1']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.1 ]
thetas['thetas_0.2']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.2 ]
thetas['thetas_0.3']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.3 ]
thetas['thetas_0.4']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.4 ]
thetas['thetas_0.5']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.5 ]
thetas['thetas_0.6']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.6 ]
thetas['thetas_0.7']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.7 ]
thetas['thetas_0.8']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.8 ]
thetas['thetas_0.9']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.9 ]
thetas['thetas_1.0']       =  [-1.0,  0.2,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  1.0 ]

#thetas['thetas_11']       =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.4 ]
#thetas['thetas_12']       =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.45 ]

filename = 'mock_thetas'+iot_number+'.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_mock+filename, 'wb') as f:
    pickle.dump(thetas, f)

thetas = {}

thetas['parnames']        =  ['logzsol', 'dust2', 'logmass', 'logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios', 'dust_index', 'dust1_fraction', 'igm_factor', 'gas_logz', 'gas_logu', 'frac_obrun']
thetas['thetas_original'] =  [-0.5,  0.6, 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. ,  0. , -2. ,  0. ]
thetas['thetas_35']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.35 ]
thetas['thetas_40']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.4 ]
thetas['thetas_45']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.45]
thetas['thetas_50']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.5 ]
thetas['thetas_55']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.55 ]
thetas['thetas_60']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.6 ]
thetas['thetas_65']        =  [-1.,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.65 ]


filename = 'mock_thetas'+iot_number+'.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_mock+filename, 'wb') as f:
    pickle.dump(thetas, f)

## Build
- build obs with galaxy 3 because it has all the bands and all the emission lines and it doesnt matter which one
- build model
- build sps

In [6]:
thetas.keys()

dict_keys(['parnames', 'thetas_original', 'thetas_0.0', 'thetas_0.1', 'thetas_0.2', 'thetas_0.3', 'thetas_0.4', 'thetas_0.5', 'thetas_0.6', 'thetas_0.7', 'thetas_0.8', 'thetas_0.9', 'thetas_1.0'])

In [7]:
objid   =  3
obs     =  Build.build_obs(objid=objid)
model   =  Build.build_model(objid=objid, fit_el=True)
sps     =  Build.build_sps()

GALAXIE: J011309+000223, ID: 3, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
GALAXIE: J011309+000223, ID: 3, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
SFH template = continuity_sfh
redshift init = 0.3061555
FITTING EL MODEL


## Predict 
- spec and phot predicted for the different thetas, which vary frac_obrun from 0, 0.1, 0.3, 0.5, 0.7, 0.9, 1

In [8]:
spec_00, phot_00, _ = model.predict(thetas['thetas_0.0'], obs=obs, sps=sps)
spec_01, phot_01, _ = model.predict(thetas['thetas_0.1'], obs=obs, sps=sps)
spec_02, phot_02, _ = model.predict(thetas['thetas_0.2'], obs=obs, sps=sps)
spec_03, phot_03, _ = model.predict(thetas['thetas_0.3'], obs=obs, sps=sps)
spec_04, phot_04, _ = model.predict(thetas['thetas_0.4'], obs=obs, sps=sps)
spec_05, phot_05, _ = model.predict(thetas['thetas_0.5'], obs=obs, sps=sps)
spec_06, phot_06, _ = model.predict(thetas['thetas_0.6'], obs=obs, sps=sps)
spec_07, phot_07, _ = model.predict(thetas['thetas_0.7'], obs=obs, sps=sps)
spec_08, phot_08, _ = model.predict(thetas['thetas_0.8'], obs=obs, sps=sps)
spec_09, phot_09, _ = model.predict(thetas['thetas_0.9'], obs=obs, sps=sps)
spec_10, phot_10, _ = model.predict(thetas['thetas_1.0'], obs=obs, sps=sps)

#spec_12, phot_12, _ = model.predict(thetas['thetas_12'], obs=obs, sps=sps)
#spec_11, phot_11, _ = model.predict(thetas['thetas_11'], obs=obs, sps=sps)


## Change the SNR 
- the snr can not be larger than 20

In [9]:
snr_el_corrected    = []
snr_phot_corrected  = []

for i in snr_el:
    if i<20:
        snr_el_corrected.append(i)
    else:
        snr_el_corrected.append(20)

for i in snr_phot:
    if i<20:
        snr_phot_corrected.append(i)
    else:
        snr_phot_corrected.append(20)

## Generate the Uncertainties

In [10]:
snr_p = [20 for i in snr_phot_corrected]
snr_s = [20 for i in snr_el_corrected]

In [11]:
spec_00_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_00/snr_s))
spec_01_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_01/snr_s))
spec_02_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_02/snr_s))
spec_03_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_03/snr_s))
spec_04_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_04/snr_s))
spec_05_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_05/snr_s))
spec_06_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_06/snr_s))
spec_07_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_07/snr_s))
spec_08_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_08/snr_s))
spec_09_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_09/snr_s))
spec_10_dis     =   np.random.normal(loc = 0., scale = np.abs(spec_10/snr_s))

phot_00_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_00/snr_p))
phot_01_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_01/snr_p))
phot_02_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_02/snr_p))
phot_03_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_03/snr_p))
phot_04_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_04/snr_p))
phot_05_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_05/snr_p))
phot_06_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_06/snr_p))
phot_07_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_07/snr_p))
phot_08_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_08/snr_p))
phot_09_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_09/snr_p))
phot_10_dis     =   np.random.normal(loc = 0., scale = np.abs(phot_10/snr_p))

In [12]:
spec_00_unc    =   np.abs(spec_00/snr_s)
spec_01_unc    =   np.abs(spec_01/snr_s)
spec_02_unc    =   np.abs(spec_02/snr_s)
spec_03_unc    =   np.abs(spec_03/snr_s)
spec_04_unc    =   np.abs(spec_04/snr_s)
spec_05_unc    =   np.abs(spec_05/snr_s)
spec_06_unc    =   np.abs(spec_06/snr_s)
spec_07_unc    =   np.abs(spec_07/snr_s)
spec_08_unc    =   np.abs(spec_08/snr_s)
spec_09_unc    =   np.abs(spec_09/snr_s)
spec_10_unc    =   np.abs(spec_10/snr_s)

phot_00_unc    =   np.abs(phot_00/snr_p)
phot_01_unc    =   np.abs(phot_01/snr_p)
phot_02_unc    =   np.abs(phot_02/snr_p)
phot_03_unc    =   np.abs(phot_03/snr_p)
phot_04_unc    =   np.abs(phot_04/snr_p)
phot_05_unc    =   np.abs(phot_05/snr_p)
phot_06_unc    =   np.abs(phot_06/snr_p)
phot_07_unc    =   np.abs(phot_07/snr_p)
phot_08_unc    =   np.abs(phot_08/snr_p)
phot_09_unc    =   np.abs(phot_09/snr_p)
phot_10_unc    =   np.abs(phot_10/snr_p)


## Generate the Data
- combine phot with the generated distortion
- take the distortion as the uncertainty
- set the masks to True everywhere
- take the same data than for the galaxies for
    - filters, wave_effective, wavelength, line_ind, cat_row, id, redshift and line_names

In [13]:
dis_data = {}

index        =  ['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']
phot_all     =  [phot_00, phot_01, phot_02, phot_03, phot_04, phot_05, phot_06, phot_07, phot_08, phot_09, phot_10]
phot_all_unc =  [phot_00_unc, phot_01_unc, phot_02_unc, phot_03_unc, phot_04_unc, phot_05_unc, phot_06_unc, phot_07_unc, phot_08_unc, phot_09_unc, phot_10_unc]
phot_all_dis =  [phot_00_dis, phot_01_dis, phot_02_dis, phot_03_dis, phot_04_dis, phot_05_dis, phot_06_dis, phot_07_dis, phot_08_dis, phot_09_dis, phot_10_dis]
spec_all     =  [spec_00, spec_01, spec_02, spec_03, spec_04, spec_05, spec_06, spec_07, spec_08, spec_09, spec_10]
spec_all_unc =  [spec_00_unc, spec_01_unc, spec_02_unc, spec_03_unc, spec_04_unc, spec_05_unc, spec_06_unc, spec_07_unc, spec_08_unc, spec_09_unc, spec_10_unc]
spec_all_dis =  [spec_00_dis, spec_01_dis, spec_02_dis, spec_03_dis, spec_04_dis, spec_05_dis, spec_06_dis, spec_07_dis, spec_08_dis, spec_09_dis, spec_10_dis]



for i, p in zip(index, range(len(phot_all))):
    dis_data['thetas_'+i]      =   thetas['thetas_'+i]
    dis_data['maggies_'+i]     =   phot_all[p] + phot_all_dis[p]
    dis_data['maggies_unc_'+i] =   phot_all_unc[p]
    dis_data['spec_'+i]        =   spec_all[p] + spec_all_dis[p]
    dis_data['spec_unc_'+i]    =   spec_all_unc[p]

dis_data['filters']         =   obs['filters']
dis_data['wave_effective']  =   obs['wave_effective']
dis_data['wavelength']      =   obs['wavelength']
dis_data['mask']            =   [True for i in dis_data['wavelength']]
dis_data['phot_mask']       =   [True for i in dis_data['filters']]
dis_data['line_ind']        =   obs['line_ind']
dis_data['cat_row']         =   obs['cat_row']
dis_data['id']              =   obs['id']
dis_data['z_spec']          =   obs['z_spec']
dis_data['line_names']      =   obs['line_names']

dis_data = {}


index        =  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
phot_all     =  [phot_0, phot_1, phot_2, phot_3, phot_4, phot_5, phot_6, phot_7, phot_8, phot_9, phot_10, phot_11, phot_12]
phot_all_unc =  [phot_0_unc, phot_1_unc, phot_2_unc, phot_3_unc, phot_4_unc, phot_5_unc, phot_6_unc, phot_7_unc , phot_8_unc , phot_9_unc , phot_10_unc, phot_11_unc, phot_12_unc]
phot_all_dis =  [phot_0_dis, phot_1_dis, phot_2_dis, phot_3_dis, phot_4_dis, phot_5_dis, phot_6_dis, phot_7_dis , phot_8_dis , phot_9_dis , phot_10_dis, phot_11_dis, phot_12_dis]
spec_all     =  [spec_0, spec_1, spec_2, spec_3, spec_4, spec_5, spec_6, spec_7, spec_8, spec_9, spec_10, spec_11, spec_12]
spec_all_unc =  [spec_0_unc, spec_1_unc, spec_2_unc, spec_3_unc, spec_4_unc, spec_5_unc, spec_6_unc, spec_7_unc , spec_8_unc , spec_9_unc , spec_10_unc, spec_11_unc, spec_12_unc]
spec_all_dis =  [spec_0_dis, spec_1_dis, spec_2_dis, spec_3_dis, spec_4_dis, spec_5_dis, spec_6_dis, spec_7_dis , spec_8_dis , spec_9_dis , spec_10_dis, spec_11_dis, spec_12_dis]

for i, p in zip(index, range(len(phot_all))):
    dis_data['thetas_'+i]      =   thetas['thetas_'+i]
    dis_data['maggies_'+i]     =   phot_all[p] + phot_all_dis[p]
    dis_data['maggies_unc_'+i] =   phot_all_unc[p]
    dis_data['spec_'+i]        =   spec_all[p] + spec_all_dis[p]
    dis_data['spec_unc_'+i]    =   spec_all_unc[p]

dis_data['filters']         =   obs['filters']
dis_data['wave_effective']  =   obs['wave_effective']
dis_data['wavelength']      =   obs['wavelength']
dis_data['mask']            =   [True for i in dis_data['wavelength']]
dis_data['phot_mask']       =   [True for i in dis_data['filters']]
dis_data['line_ind']        =   obs['line_ind']
dis_data['cat_row']         =   obs['cat_row']
dis_data['id']              =   obs['id']
dis_data['z_spec']          =   obs['z_spec']
dis_data['line_names']      =   obs['line_names']

## Save the Distorted Data in Pickle:

In [14]:
filename = path_data + 'distorted_data'+iot_number+'.pickle'

with open(filename, 'wb') as f:
    pickle.dump(dis_data, f)


The distorted data is now used as the new obs dictionary in the next step, which is putting everything into prospector again. For that we use DISPRO.py.

In [15]:
dis_data

{'thetas_0.0': [-1.0,
  0.6,
  8.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  -1.0,
  -2.0,
  0.0],
 'maggies_0.0': array([5.30431468e-12, 1.13311265e-11, 2.79489563e-11, 3.19004852e-11,
        4.42147575e-11, 1.61833691e-12, 3.08000383e-12]),
 'maggies_unc_0.0': array([2.63126505e-13, 5.76590595e-13, 1.46339242e-12, 1.75917227e-12,
        2.39035607e-12, 7.44934204e-14, 1.52558133e-13]),
 'spec_0.0': array([3.02027013e-19, 4.35863248e-19, 5.28832654e-19, 1.02659303e-19,
        1.51462180e-19, 1.73275822e-19, 2.88156013e-19, 5.83808705e-19,
        9.34794278e-20, 1.34930524e-18, 1.90555991e-18, 6.24695551e-18,
        2.11184392e-19, 2.30625533e-20, 1.94609281e-20, 6.13597534e-18,
        5.23113256e-20, 1.25455783e-19, 9.57575200e-20]),
 'spec_unc_0.0': array([1.53541494e-20, 2.11110740e-20, 2.53153527e-20, 4.91210311e-21,
        7.93693087e-21, 9.05691050e-21, 1.54190016e-20, 3.01941656e-20,
        4.94987606e-21, 7.45694977e-20, 9.70449409e-20, 2.

In [16]:
dis_data['thetas_1.0']

[-1.0,
 0.2,
 8.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 -1.0,
 -2.0,
 1.0]

In [18]:
dis_data.keys()

dict_keys(['thetas_0.0', 'maggies_0.0', 'maggies_unc_0.0', 'spec_0.0', 'spec_unc_0.0', 'thetas_0.1', 'maggies_0.1', 'maggies_unc_0.1', 'spec_0.1', 'spec_unc_0.1', 'thetas_0.2', 'maggies_0.2', 'maggies_unc_0.2', 'spec_0.2', 'spec_unc_0.2', 'thetas_0.3', 'maggies_0.3', 'maggies_unc_0.3', 'spec_0.3', 'spec_unc_0.3', 'thetas_0.4', 'maggies_0.4', 'maggies_unc_0.4', 'spec_0.4', 'spec_unc_0.4', 'thetas_0.5', 'maggies_0.5', 'maggies_unc_0.5', 'spec_0.5', 'spec_unc_0.5', 'thetas_0.6', 'maggies_0.6', 'maggies_unc_0.6', 'spec_0.6', 'spec_unc_0.6', 'thetas_0.7', 'maggies_0.7', 'maggies_unc_0.7', 'spec_0.7', 'spec_unc_0.7', 'thetas_0.8', 'maggies_0.8', 'maggies_unc_0.8', 'spec_0.8', 'spec_unc_0.8', 'thetas_0.9', 'maggies_0.9', 'maggies_unc_0.9', 'spec_0.9', 'spec_unc_0.9', 'thetas_1.0', 'maggies_1.0', 'maggies_unc_1.0', 'spec_1.0', 'spec_unc_1.0', 'filters', 'wave_effective', 'wavelength', 'mask', 'phot_mask', 'line_ind', 'cat_row', 'id', 'z_spec', 'line_names'])